# Binary classification with Keras Sequential model

In this notebook we will build a binar classification model utilizing the [Keras Sequential model](https://www.tensorflow.org/guide/keras/sequential_model).

The sequential API is well suited for building models with layers that proceed in a sequential manner. This entails that the order of the layers matters.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from simple_keras import KerasSeqClassifier

## Load data

We start by loading the `MNIST` dataset which contains images depicting hand-written numbers from 0-9.

In [ ]:
# Split
split = ['train', 'test']

# Load cats_vs_dogs dataset
(ds_train, ds_test), ds_info = tfds.load(
    name='mnist',
    split=split,
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

## Inspect dataset

Start by inspecting the data. A TensorFlow `Dataset` has a lot of properties, for reference see [TensorFlow Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Datasethttps://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
# Print info
print(ds_info.splits['train'].num_examples)
print(ds_info.splits['test'].num_examples)
print(ds_test.element_spec)

# Calculate number of classes in label
num_classes = ds_info.features['label'].num_classes
print(num_classes)

We can visualize some of the images.

In [ ]:
images_iter = iter(x_train for x_train, _ in ds_train.take(9).cache().repeat())
labels_iter = iter(y_train for _, y_train in ds_train.take(9).cache().repeat())
plt.figure(figsize=(8, 8))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(next(images_iter).numpy().astype("uint8"))
    plt.title(next(labels_iter).numpy().astype("uint8"))
    plt.axis("off")

## Preprocess data

Before training we need to preprocess the images.

We start by normalizing the image pixels.

In [ ]:
def normalize(image, label):
    """Normalize image pixels."""
    return tf.cast(image, tf.float32) / 255.0, label

In [ ]:
# Normalize train and test data
ds_train = ds_train.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Print element spec after preprocessing
print(ds_train.element_spec)
print(ds_test.element_spec)

The shapes are the same, `(28, 28, 1)`, but the type has changed, `dtype=tf.float32`.

The `Dataset` API has some nice functionalities for speeding up the traning time.

By caching the datasets we we will save some operations (like file opening and data reading) from being executed during each epoch, [reference](https://www.tensorflow.org/guide/data_performance#caching).

### Set up training data

First, we take the training data and apply the following steps:
* cache it before shuffling for better performance
* for true randomness, set shuffle buffer to full dataset size
* batch elements of the dataset after shuffling to get unique batches at each epoch
* prefetch to increase performace

In [ ]:
# Cache train data
ds_train = ds_train.cache()

# Shuffle data for true randomness and to reduce memory usage
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)

# Set batch size
ds_train = ds_train.batch(32)

# Prefetch
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


Second, we take our evaluation data. We do similar steps but skip a few:
* we don't need to shuffle the data
* caching is done after batching because batches can be the same between epochs

In [ ]:

# Set batch size
ds_test = ds_test.batch(32)

# Cache test data
ds_test = ds_test.cache()

# Prefetch
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Train model

Next we instantiate the model and let it train for a number of epochs.

In [ ]:
# Instantiate classifier
clf = KerasSeqClassifier(input_shape=(28, 28, 1))

In [ ]:
# Train model
clf.call(ds_train=ds_train, ds_eval=ds_test, epochs=5)

In [ ]:
clf.model.evaluate(ds_test)